### Data Cleaning Overview
- Remove duplicates.
- Remove irrelevant data.
- Convert data type.
- Standardise formatting.
- Handle missing values.
- Handle outlier values.

In [7]:
import pandas as pd

In [8]:
df_hdb = pd.read_csv('../data/ResaleFlatPrices/ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv')
df_hdb

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0
...,...,...,...,...,...,...,...,...,...,...,...
37148,2016-12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,82,488000.0
37149,2016-12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,69,455000.0
37150,2016-12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,74,778000.0
37151,2016-12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,70,575000.0


#### Convert new column `address` from `block` and `street_name`

In [9]:
df_hdb['address'] = df_hdb['block'] + " " + df_hdb['street_name']
df_hdb

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,541 ANG MO KIO AVE 10
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,163 ANG MO KIO AVE 4
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,446 ANG MO KIO AVE 10
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,557 ANG MO KIO AVE 10
...,...,...,...,...,...,...,...,...,...,...,...,...
37148,2016-12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,82,488000.0,297 YISHUN ST 20
37149,2016-12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,69,455000.0,838 YISHUN ST 81
37150,2016-12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,74,778000.0,664 YISHUN AVE 4
37151,2016-12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,70,575000.0,325 YISHUN CTRL


### Convert addresses to lat/lng coordinates
Purpose: To use coordinates to calculate distance

How: Use OneMapSG API to convert address to coordinates

In [10]:
import requests

def get_geocoordinates_from_address(address: str) -> tuple:
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={address}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    
    response = requests.get(url)    
    data = response.json()

    # Check if there are results
    if data['results']:
        first_result = data['results'][0]
        latitude = first_result['LATITUDE']
        longitude = first_result['LONGITUDE']
        return (latitude, longitude)
    else:
        return ("NA", "NA")

In [11]:
address_list = df_hdb['address'].unique().tolist()
len(address_list)

7921

In [12]:
latitude_list = longitude_list = []

count = 1
for address in address_list:
    try:
        geocoordinates = get_geocoordinates_from_address(address)
        # print(count, geocoordinates)
    except ConnectionError:
        geocoordinates = ("TBC", "TBC")
   
    latitude, longitude = geocoordinates
    
    latitude_list.append(latitude)
    longitude_list.append(longitude)
    count += 1

Due to an initial error of declaring `latitude_list = longtitude_list = []`, both lists now have latitude and longtitude values. Since it took around 20min30s to query the coordinates, we shan't repeat the entire process but allocate the latitudes and longtitudes to the respective lists by recognising that every even index (starting from 0th) is the latitude and every odd index (start from 1st) is the longtitude

In [13]:
# convert latitude list to contain the eventh index
latitude_list = latitude_list[::2]
# convert longtitude list to contain the odd index
longitude_list = longitude_list[1::2]
# check if both have the length of 7921
print(len(latitude_list), len(longitude_list))

7921 7921


In [14]:
df_hdb_coordinates = pd.DataFrame({'address': address_list, 'latitude': latitude_list, 'longitude': longitude_list})
df_hdb_coordinates.head(5)

,address,latitude,longitude
0,174 ANG MO KIO AVE 4,1.37509746867904,103.83761896123
1,541 ANG MO KIO AVE 10,1.37392239168826,103.855621371068
2,163 ANG MO KIO AVE 4,1.37355178595963,103.838168959329
3,446 ANG MO KIO AVE 10,1.36776094720351,103.85535715026
4,557 ANG MO KIO AVE 10,1.3716257020332,103.857736107527


In [15]:
df_hdb_coordinates.to_csv('../data/modified/hdb_coordinates.csv')

In [16]:
df_hdb.to_csv('../data/modified/hdb_cleaned_data.csv')

In [17]:
df_hdb_coordinates = pd.read_csv('../data/modified/hdb_coordinates.csv')
df_hdb = pd.read_csv('../data/modified/hdb_cleaned_data.csv')

merged_df = pd.merge(df_hdb, df_hdb_coordinates, on='address', how='left')
merged_df.head(10)

,Unnamed: 0_x,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,Unnamed: 0_y,latitude,longitude
0,0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4,0,1.375097,103.837619
1,1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,541 ANG MO KIO AVE 10,1,1.373922,103.855621
2,2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,163 ANG MO KIO AVE 4,2,1.373552,103.838169
3,3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,446 ANG MO KIO AVE 10,3,1.367761,103.855357
4,4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,557 ANG MO KIO AVE 10,4,1.371626,103.857736
5,5,2015-01,ANG MO KIO,3 ROOM,603,ANG MO KIO AVE 5,07 TO 09,67.0,New Generation,1980,64,290000.0,603 ANG MO KIO AVE 5,5,1.380201,103.835756
6,6,2015-01,ANG MO KIO,3 ROOM,709,ANG MO KIO AVE 8,01 TO 03,68.0,New Generation,1980,64,290000.0,709 ANG MO KIO AVE 8,6,1.371137,103.847662
7,7,2015-01,ANG MO KIO,3 ROOM,333,ANG MO KIO AVE 1,01 TO 03,68.0,New Generation,1981,65,293000.0,333 ANG MO KIO AVE 1,7,1.361343,103.851699
8,8,2015-01,ANG MO KIO,3 ROOM,109,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,62,300000.0,109 ANG MO KIO AVE 4,8,1.370097,103.837688
9,9,2015-01,ANG MO KIO,3 ROOM,564,ANG MO KIO AVE 3,13 TO 15,68.0,New Generation,1985,69,307500.0,564 ANG MO KIO AVE 3,9,1.369848,103.859404


In [18]:
merged_df.drop(columns='Unnamed: 0_y')
merged_df.to_csv('../data/modified/hdb_transaction_coordinates.csv')

### Cleaning the `mrt_stations.csv` dataset

We will remove the rows where the MRT stations have yet to be opened

In [30]:
# load hdb_coordinates.csv
mrt_stations = pd.read_csv('../data/modified/mrt_stations.csv')
print(mrt_stations.head())
mrt_stations.shape

           mrt_station_name  latitude   longitude      opening_date
0   Jurong East MRT Station  1.333333  103.742222   5 November 1988
1   Bukit Batok MRT Station  1.349167  103.749722     10 March 1990
2  Bukit Gombak MRT Station  1.358611  103.751667     10 March 1990
3     Brickland MRT Station  1.368611  103.749444    Not opened yet
4       Yew Tee MRT Station  1.396986  103.747239  10 February 1996


(185, 4)

In [31]:
# remove rows that have "Not opened yet" as values in the opening_date column
mrt_stations = mrt_stations[mrt_stations['opening_date'] != 'Not opened yet'].reset_index(drop=True)
print(mrt_stations.head())
mrt_stations.shape

           mrt_station_name  latitude   longitude      opening_date
0   Jurong East MRT Station  1.333333  103.742222   5 November 1988
1   Bukit Batok MRT Station  1.349167  103.749722     10 March 1990
2  Bukit Gombak MRT Station  1.358611  103.751667     10 March 1990
3       Yew Tee MRT Station  1.396986  103.747239  10 February 1996
4        Kranji MRT Station  1.425047  103.761853  10 February 1996


(129, 4)

In [32]:
# convert mrt_stations into a csv file
mrt_stations.to_csv('../data/modified/mrt_stations_opened.csv', index=False)

### Cleaning the BTO Supply dataset

In [33]:
# load bto supply dataset
bto_supply = pd.read_csv("../data/BTO Supply Data (2004-2024).csv")
print(bto_supply.shape)
bto_supply.head(15)

(445, 7)


,Town name,BTO name . . . . . .,Launch,Estimated,Number,Brochure,Unnamed: 6
0,NaN,NaN,date,completion,of units,(note),NaN
1,NaN,NaN,NaN,date (note),NaN,NaN,NaN
2,Yishun,Acacia Breeze,24-Nov-11,1Q 2015,834,NaN,Info Plans
3,Woodlands,Admiralty Flora,22-May-14,2Q 2017,402,Brochure,Info Plans
4,Woodlands,Admiralty Grove,26-Nov-13,3Q 2017,993,Brochure,Info Plans
5,Bukit Merah,Alexandra Peaks,5-Dec-23,59 months,904,Brochure,NaN
6,Bukit Merah,Alexandra Vale (PLH),30-Aug-22,2Q 2028,782,Brochure,NaN
7,Bidadari,Alkaff Breeze,4-Feb-21,2Q 2024,353,Brochure,NaN
8,Bidadari,Alkaff CourtView,17-Nov-15,2Q 2020,1258,Brochure,NaN
9,Bidadari,Alkaff LakeView,17-Nov-15,4Q 2019,531,Brochure,NaN


In [34]:
# drop the first 2 rows
bto_supply = bto_supply.iloc[2:]
# also drop any rows that do not have values for Launch, Estimated and Number
bto_supply = bto_supply.dropna(subset=['Launch', 'Estimated', 'Number']).reset_index(drop=True)
print(bto_supply.shape)
bto_supply.head()

(436, 7)


,Town name,BTO name . . . . . .,Launch,Estimated,Number,Brochure,Unnamed: 6
0,Yishun,Acacia Breeze,24-Nov-11,1Q 2015,834,NaN,Info Plans
1,Woodlands,Admiralty Flora,22-May-14,2Q 2017,402,Brochure,Info Plans
2,Woodlands,Admiralty Grove,26-Nov-13,3Q 2017,993,Brochure,Info Plans
3,Bukit Merah,Alexandra Peaks,5-Dec-23,59 months,904,Brochure,NaN
4,Bukit Merah,Alexandra Vale (PLH),30-Aug-22,2Q 2028,782,Brochure,NaN


We will be using the launch dates later on, we shall standardise the format to be DD-MMM-YY. For entries that do have any days (DD) we shall standardise it to the first of the month. 

In [35]:
# convert the launch column to the format DD-MMM-YY
def convert_launch_date(launch_date):
    launch_date = launch_date.split('-')
    # print(launch_date)
    if len(launch_date) == 1: # to handle dates like Dec 2004
        launch_date = launch_date[0].split('\xa0')
        month = launch_date[0]
        year = launch_date[1][2:]
        launch_date = '01-' + month + '-' + year
    elif len(launch_date) == 2:
        launch_date = '01-' + launch_date[0] + '-' + launch_date[1]
    else:
        day = launch_date[0]
        month = launch_date[1]
        year = launch_date[2]
        if len(day) == 1:
            day = '0' + day
        launch_date = day + '-' + month + '-' + year
    return launch_date

print(convert_launch_date('2-May-04'))
print(convert_launch_date('May-04'))
print(convert_launch_date('23-May-04'))
print(convert_launch_date('Dec 2003'))

02-May-04
01-May-04
23-May-04
01-Dec-03


In [36]:
# apply convert_launch_date to the launch column
bto_supply['Launch'] = bto_supply['Launch'].apply(convert_launch_date)
bto_supply.head()

,Town name,BTO name . . . . . .,Launch,Estimated,Number,Brochure,Unnamed: 6
0,Yishun,Acacia Breeze,24-Nov-11,1Q 2015,834,NaN,Info Plans
1,Woodlands,Admiralty Flora,22-May-14,2Q 2017,402,Brochure,Info Plans
2,Woodlands,Admiralty Grove,26-Nov-13,3Q 2017,993,Brochure,Info Plans
3,Bukit Merah,Alexandra Peaks,05-Dec-23,59 months,904,Brochure,NaN
4,Bukit Merah,Alexandra Vale (PLH),30-Aug-22,2Q 2028,782,Brochure,NaN


In [37]:
# check if number column has any non-numeric values
bto_supply[~bto_supply['Number'].str.isnumeric()]

,Town name,BTO name . . . . . .,Launch,Estimated,Number,Brochure,Unnamed: 6


Now we add the respective BTO coordinates based on `Town name` and `BTO name`, but before that we must update the BTO name column first

In [38]:
# update BTO name column 
bto_supply['BTO name'] = bto_supply['BTO name . . . . . .'].str.replace('BTO', 'BTO name . . . . . .')
# drop the prev column 
bto_supply = bto_supply.drop(columns=['BTO name . . . . . .'])
bto_supply.head()

,Town name,Launch,Estimated,Number,Brochure,Unnamed: 6,BTO name
0,Yishun,24-Nov-11,1Q 2015,834,NaN,Info Plans,Acacia Breeze
1,Woodlands,22-May-14,2Q 2017,402,Brochure,Info Plans,Admiralty Flora
2,Woodlands,26-Nov-13,3Q 2017,993,Brochure,Info Plans,Admiralty Grove
3,Bukit Merah,05-Dec-23,59 months,904,Brochure,NaN,Alexandra Peaks
4,Bukit Merah,30-Aug-22,2Q 2028,782,Brochure,NaN,Alexandra Vale (PLH)


Remove (PLH) from BTO name

In [39]:
# remove (PLH) from the BTO name    
bto_supply['BTO name'] = bto_supply['BTO name'].str.replace(' (PLH)', '')

Then we combine the `Town name` and `BTO name` for querying the BTO later

In [40]:
# create new column for BTO
bto_supply['BTO'] = bto_supply['Town name'] + ' ' + bto_supply['BTO name']
bto_supply.head()

,Town name,Launch,Estimated,Number,Brochure,Unnamed: 6,BTO name,BTO
0,Yishun,24-Nov-11,1Q 2015,834,NaN,Info Plans,Acacia Breeze,Yishun Acacia Breeze
1,Woodlands,22-May-14,2Q 2017,402,Brochure,Info Plans,Admiralty Flora,Woodlands Admiralty Flora
2,Woodlands,26-Nov-13,3Q 2017,993,Brochure,Info Plans,Admiralty Grove,Woodlands Admiralty Grove
3,Bukit Merah,05-Dec-23,59 months,904,Brochure,NaN,Alexandra Peaks,Bukit Merah Alexandra Peaks
4,Bukit Merah,30-Aug-22,2Q 2028,782,Brochure,NaN,Alexandra Vale,Bukit Merah Alexandra Vale


In [29]:
# export bto_supply to a csv file
bto_supply.to_csv('../data/modified/bto_supply_cleaned.csv', index=False)